Notebook to scrape news article 

In [53]:
import os
directory = r'your_dir'

# Change the current working directory to the specified directory
os.chdir(directory)

In [44]:
#pip install newspaper3k lxml[html_clean]

In [46]:
#pip install requests beautifulsoup4 newspaper3k lxml[html_clean]


In [48]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
from datetime import datetime
import time

def fetch_article_links(news_url):
    response = requests.get(news_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = []
    for a_tag in soup.find_all('a', href=True):
        link = a_tag['href']
        if link.startswith('http'):
            links.append(link)
        elif link.startswith('/'):
            links.append(news_url + link)
    return links

def get_articles_from_links(links, output_file, keywords, start_date, end_date):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    with open(output_file, 'w', encoding='utf-8') as file:
        for link in links:
            if "cnn-underscored" in link or "cnn.com" not in link:
                continue  # Skip problematic sections or invalid links
            try:
                # Use requests to get the article content
                response = requests.get(link, headers=headers)
                response.raise_for_status()

                # Parse the article with newspaper
                article = Article(link)
                article.set_html(response.text)
                article.parse()
                article_date = article.publish_date

                if article_date is None:
                    continue
                if article_date.tzinfo is not None:
                    article_date = article_date.replace(tzinfo=None)
                if not (start_date <= article_date <= end_date):
                    continue
                if not any(keyword.lower() in article.text.lower() for keyword in keywords):
                    continue

                file.write(f"Title: {article.title}\n")
                file.write(f"URL: {article.url}\n")
                file.write(f"Date: {article_date.strftime('%Y-%m-%d')}\n")
                file.write(f"Text: {article.text}\n\n")
                print(f"Processed article: {link}")
            except Exception as e:
                print(f"Failed to process article: {link}, Error: {str(e)}")
            time.sleep(1)  # Add a delay to prevent rate limiting

news_url = 'http://cnn.com'
output_file = 'filtered_news_articles.txt'
keywords = ["OpenAI", "ChatGPT", "GPT-3", "GPT-4"]
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 7, 31)

links = fetch_article_links(news_url)
print(f"Total articles found: {len(links)}")

get_articles_from_links(links, output_file, keywords, start_date, end_date)

print(f"Filtered articles have been saved to {output_file}")


Total articles found: 547


KeyboardInterrupt: 